In [1]:
from mikeio1d import Res1D

# Test read geometry for reaches / nodes

In [2]:
res = Res1D(r'..\tests\testdata\Network.res1d')
res

<mikeio1d.Res1D>
Start time: 1994-08-07 16:35:00
End time: 1994-08-07 18:35:00
# Timesteps: 110
# Catchments: 0
# Nodes: 119
# Reaches: 118
# Globals: 0
0 - WaterLevel <m>
1 - Discharge <m^3/s>

## Get sample node and reach

In [3]:
node = res.nodes['1']
node

In [4]:
reach = res.reaches['109l1']
reach

## Print coordinates of points on node

In [5]:
print(node.get_XCoordinate(), node.get_YCoordinate())

-687934.6000976562 -1056500.69921875


## Print coordinates of points along reach

In [6]:
dpoints = list(reach.get_DigiPoints())
dpoints

In [7]:
for dpoint in dpoints:
    print(dpoint.get_X(), dpoint.get_Y())

-687740.1002807617 -1056132.1002807617
-687766.4008789062 -1056138.6010131836


# Test read geometry for catchments

In [8]:
res = Res1D(r'..\tests\testdata\Catchments.res1d')
res

<mikeio1d.Res1D>
Start time: 1994-08-07 16:35:00
End time: 1994-08-07 18:35:00
# Timesteps: 108
# Catchments: 31
# Nodes: 0
# Reaches: 0
# Globals: 0
0 - TotalRunOff <m^3/s>
1 - ActualRainfall <m/s>
2 - ZinkLoadRR <kg/s>
3 - ZinkMassAccumulatedRR <kg>
4 - ZinkRR <mg/l>

## Get sample catchment

In [9]:
catchment = res.catchments['100_16_16']
catchment

# Print coordinates of points forming the catchment

In [10]:
shape = catchment.get_Shape()
shape

In [11]:
element = list(shape)[0]
element

In [12]:
vertices = list(element.get_Vertices())
vertices

In [13]:
for vertex in vertices:
    print(vertex.get_X(), vertex.get_Y())

-687895.1632080078 -1056385.7786865234
-687849.0208129883 -1056284.5764770508
-687900.6370849609 -1056313.5150146484
-687949.9086914062 -1056434.4193115234
-687923.3170776367 -1056450.6895141602
-687895.1632080078 -1056385.7786865234


# Export to shapefile

In [14]:
# This would add a dependency
import shapefile

In [15]:
res = Res1D(r'..\tests\testdata\Network.res1d')
res

<mikeio1d.Res1D>
Start time: 1994-08-07 16:35:00
End time: 1994-08-07 18:35:00
# Timesteps: 110
# Catchments: 0
# Nodes: 119
# Reaches: 118
# Globals: 0
0 - WaterLevel <m>
1 - Discharge <m^3/s>

## Export all nodes to a shapefile

In [16]:
with shapefile.Writer('Nodes.shp') as shp:
    shp.field('id', 'C')
    for node in res.nodes:
        shp.point(res.nodes[node].get_XCoordinate(), res.nodes[node].get_YCoordinate())
        shp.record(node)

## Export all reaches to a shapefile

In [17]:
with shapefile.Writer('Reaches.shp') as shp:
    shp.field('id', 'C')
    for reach in res.reaches:
        line_points = []
        for dpoint in list(res.reaches[reach].get_DigiPoints()):
            line_points.append([dpoint.get_X(), dpoint.get_Y()])
        shp.line([line_points])
        shp.record(reach)

## Export all catchments to a shapefile

In [18]:
res = Res1D(r'..\tests\testdata\Catchments.res1d')
res

<mikeio1d.Res1D>
Start time: 1994-08-07 16:35:00
End time: 1994-08-07 18:35:00
# Timesteps: 108
# Catchments: 31
# Nodes: 0
# Reaches: 0
# Globals: 0
0 - TotalRunOff <m^3/s>
1 - ActualRainfall <m/s>
2 - ZinkLoadRR <kg/s>
3 - ZinkMassAccumulatedRR <kg>
4 - ZinkRR <mg/l>

In [19]:
with shapefile.Writer('Catchments.shp') as shp:
    shp.field('id', 'C')
    for id, catchment in res.catchments.items():
        poly_points = []
        shape = catchment.get_Shape()
        element = list(shape)[0]
        vertices = list(element.get_Vertices())
        for vertex in vertices:
            poly_points.append([vertex.get_X(), vertex.get_Y()])
        shp.poly([poly_points])
        shp.record(id)

# Clean up

In [21]:
import os

for filename in os.listdir('.'):
    if 'Catchments' in filename:
        os.remove(filename)
    if 'Nodes' in filename:
        os.remove(filename)
    if 'Reaches' in filename:
        os.remove(filename)
